<h1 align="center">
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!<br>
Brief explanation on our Booter database analysis methodology: <br>
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!</h1> 

# Analysis of: panicstresser
#### File originally retrieved from: http://4lz5rmnkd6f63tmm.onion/db/panicstresser.sql

## * Libraries that I use to analyse data

In [1]:
import pandas as pd
import numpy as np

from io import StringIO
import re

from IPython.display import display,clear_output

<h1 align="center">
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!<br>
STEP 0: Reading an input Booter database file<br>
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!</h1> 

In [2]:
dumpfile='panicstresser.sql'

<h1 align="center">
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!<br>
STEP 1: Adaptation to our Booter database schema<br>
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!</h1> 

## 1. Reading the first 'N' (100) lines of the input Booter database file

In [3]:
lines_to_read=100

with open(dumpfile) as myfile:
    firstlines=myfile.readlines()[0:lines_to_read] #put here the interval you want
    for x in firstlines:
        print(x.strip())

-- phpMyAdmin SQL Dump
-- version 3.4.10.1
-- http://www.phpmyadmin.net
--
-- Host: localhost
-- Generation Time: Jul 30, 2012 at 01:47 PM
-- Server version: 5.5.24
-- PHP Version: 5.2.6

SET SQL_MODE="NO_AUTO_VALUE_ON_ZERO";
SET time_zone = "+00:00";


/*!40101 SET @OLD_CHARACTER_SET_CLIENT=@@CHARACTER_SET_CLIENT */;
/*!40101 SET @OLD_CHARACTER_SET_RESULTS=@@CHARACTER_SET_RESULTS */;
/*!40101 SET @OLD_COLLATION_CONNECTION=@@COLLATION_CONNECTION */;
/*!40101 SET NAMES utf8 */;

--
-- Database: `panic_stresser`
--

-- --------------------------------------------------------

--
-- Table structure for table `admin-settings`
--

CREATE TABLE IF NOT EXISTS `admin-settings` (
`name` varchar(60) NOT NULL,
`value` varchar(300) NOT NULL,
UNIQUE KEY `name` (`name`)
) ENGINE=MyISAM DEFAULT CHARSET=latin1;

--
-- Dumping data for table `admin-settings`
--

INSERT INTO `admin-settings` (`name`, `value`) VALUES
('boot_status', 'online'),
('cur_url', 'http://panicboot.info/'),
('paypal_email', '');


## 2. Listing tables that have content inserted into the dump file

In [4]:
def list_tables_with_insert(dumpfile):
    tables = []
    with open(dumpfile, 'rb') as f:
        for line in f:
            line = line.decode("utf-8").strip()
            if line.lower().startswith('insert'):
                table = re.findall(r'`(.*?)`', line)
                tables.append(table[0])
    return tables

In [5]:
tables=enumerate(list_tables_with_insert(dumpfile))

for i, item in tables:
    print(i+1,item)

1 admin-settings
2 boot-log
3 friend-list
4 memberships
5 news
6 paypal-purchases
7 shells
8 specialmemberships
9 users
10 users


## How the list above looks like? 

<h2 align='center'>======================================================================
If NOT well-formed SQL dump file then you must first do the following:
======================================================================</h2>

## 3. Removing lines that are not part of the actual content to be analysed

## 4. Splitting and naming tables and columns

<h2 align='center'>======================================================================
Additional functions
======================================================================</h2>

## * Function to read tables from wel-formed SQL database dump

In [89]:
# For well formed SQL database dumps!
def read_inserted_table(dumpfile, target_table):
    sio = StringIO()
    fast_forward = True
    already_header = False
    with open(dumpfile, 'rb') as f:
        for line in f:
            line = line.decode("utf-8").strip()
            line = re.sub("(?!(([^']*'){2})*[^']*$)\)", '',line) #Step0:remove any ")" from the content of columns
            if line.lower().startswith('insert') and target_table in line:
                fast_forward = False
                if already_header:
                    continue
            if fast_forward:
                continue
            
            data = re.findall('\([^\)]*\)', line) #Step1: get the content between parentesis (i.e., insert line)
            try:
                newline = data[0].strip('()') #Step2:remove parenthesis
                newline=newline.replace('`','') #Step3: remove ` (usually in table names)
                newline=re.sub(r'(?!(([^\']*\'){2})*[^\']*$),','', newline) #Step4: remove commas from the content of columns
                newline=newline.replace('\'','') #Step5: remove single quotes
                newline=newline.replace(', ', ',') #Step6: remove single spaces after comma (i.e., in the beginning of a column)
                sio.write(newline)
                sio.write("\n")
            except IndexError:
                pass
            if line.endswith(';'):
                fast_forward = True
                already_header = True
    sio.seek(0)
    return sio

## * Converter functions for formatting content of columns
Attention, it is better to perform the conversion based on a series than a string (http://stackoverflow.com/questions/42462906/pandas-read-csv-converters-performance-issue)

In [90]:
def tolowercase(series):
    return series.str.lower()

In [91]:
def strip_singlequote(series):
    return series.str.strip('\'')

In [93]:
def timestamp2datetime(series):
    return  pd.to_datetime(series,unit='s',errors='coerce')

In [94]:
def datetimestring2datetime(series):
    return  pd.to_datetime(series, format='%d-%m-%Y %H:%M',errors='coerce')  

In [95]:
def datetimeinvertedstring2datetime(series):
    return pd.to_datetime(series, format='%Y-%m-%d %H:%M:%S',errors='coerce')

In [96]:
def datetimeampm2datetime(series):
    return  pd.to_datetime(series,format='%d-%m-%Y %I:%M:%S %p',errors='coerce')

In [97]:
def datetimemdyampm2datetime(series):
    return  pd.to_datetime(series, format='%m-%d-%Y %I:%M:%S %p',errors='coerce') 

In [98]:
def strip_parenthesisandsemicolon(series):
    return series.str.strip(');')

In [99]:
def strip_insertintologs(series):
    return int(series.str.strip('INSERT INTO `logs` VALUES ('))

In [100]:
def strip_tab(series):
    return series.str.strip('\t')

<h2 align='center'>======================================================================
Adapting EACH existing table
======================================================================</h2>

## 6.1. Table: 'admin-settings'

### How this table looks like without modification?

In [83]:
tablename='admin-settings'

pd.read_csv(read_inserted_table(dumpfile,tablename),delimiter=",",error_bad_lines=False)

,name,value
0,boot_status,online
1,cur_url,http://panicboot.info/
2,paypal_email,NaN


#### Q1: Is this table different from other previous loaded table? YES
#### Q2: Is this table similar to any table in the generic Booter database schema? YES
    - This table looks like: settings
    
#### Q3: Are there modifications required? 
    - On the table name: 
        o admin-settings -> settings
    - On the column type/converter: 
        o 
    - On the column name:
        o value -> url
        o name -> sitename
    - Split columns:
        o 
    - Add required columns:
        o siteemail

### Modifying table: 'admin-settings'

In [84]:
# Loading table with the correct name
df_settings = pd.read_csv(read_inserted_table(dumpfile, tablename),delimiter=",", error_bad_lines=False)
                        
# Applying converters on the columns of the table

# Changing names of columns
df_settings.rename(inplace=True,columns = {'value': 'url',
                                          'name':'sitename'})

# Creating empty columns (with "" [for future string] or np.nan [for future float])
df_settings['siteemail'] = ''

# Showing some lines after adapt the table
df_settings.head(10)

,sitename,url,siteemail
0,boot_status,online,
1,cur_url,http://panicboot.info/,
2,paypal_email,NaN,


## 6.2. Table: 'boot-log'

### How this table looks like without modification?

In [85]:
tablename='boot-log'

pd.read_csv(read_inserted_table(dumpfile, tablename),delimiter=",",error_bad_lines=False).head(10)

,id,uid,host,port,time,date
0,1,195,71.0.106.215,3074,210,1343616757
1,2,5,80.162.234.191,80,300,1343623684
2,3,5,80.162.234.191,80,300,1343624045
3,4,370,76.108.97.119,80,450,1343625370
4,5,262,217.23.10.201,80,300,1343625764
5,6,262,131.165.91.30,80,300,1343626141
6,7,5,168.144.98.30,80,290,1343626160
7,8,302,212.179.42.18,80,50,1343626985
8,9,1,174.24.156.177,3074,100,1343629461
9,10,395,91.48.183.141,80,290,1343629971


#### Q1: Is this table different from other previous loaded table? YES
#### Q2: Is this table similar to any table in the generic Booter database schema? YES 
    - This table looks like: attacks
    
#### Q3: Are there modifications required? 
    - On the table name: 
        o boot-log -> attacks
    - On the column type/converter: 
        o date -> timestamp2datetime()
    - On the column name:
        o uid -> userid
        o host -> targetip
        o time -> duration

    - Split columns:
        o 
    - Add required columns:
        o username
        o targeturl
        o type

### Modifying table: 'boot-log'

In [101]:
# Loading the column converting columns with predefined functions
df_attacks = pd.read_csv(read_inserted_table(dumpfile, tablename),delimiter=",",error_bad_lines=False)

# Converters                                     
df_attacks['date']=timestamp2datetime(df_attacks['date'])

# Changing names of columns
df_attacks.rename(inplace=True,columns = {'uid': 'userid',
                                         'host': 'targetip',
                                         'time': 'duration'})

# Creating empty columns (with "" [for future string] or np.nan [for future float])
df_attacks['username'] = ''
df_attacks['targeturl'] = ''
df_attacks['type'] = ''

# Showing some lines after adapt the table
df_attacks.head(10)

,id,userid,targetip,port,duration,date,username,targeturl,type
0,1,195,71.0.106.215,3074,210,2012-07-30 02:52:37,,,
1,2,5,80.162.234.191,80,300,2012-07-30 04:48:04,,,
2,3,5,80.162.234.191,80,300,2012-07-30 04:54:05,,,
3,4,370,76.108.97.119,80,450,2012-07-30 05:16:10,,,
4,5,262,217.23.10.201,80,300,2012-07-30 05:22:44,,,
5,6,262,131.165.91.30,80,300,2012-07-30 05:29:01,,,
6,7,5,168.144.98.30,80,290,2012-07-30 05:29:20,,,
7,8,302,212.179.42.18,80,50,2012-07-30 05:43:05,,,
8,9,1,174.24.156.177,3074,100,2012-07-30 06:24:21,,,
9,10,395,91.48.183.141,80,290,2012-07-30 06:32:51,,,


## 6.3. Table: 'friend-list'

### How this table looks like without modification?

In [103]:
tablename='friend-list'

pd.read_csv(read_inserted_table(dumpfile, tablename),delimiter=",",error_bad_lines=False).head(10)

,id,uid,host,name
0,7,30,68.82.250.68,BoomFireProductions
1,29,30,31.110.102.168,TehModdah
2,5,30,86.27.70.195,Cobby
3,114,78,98.213.93.116,DeathGodz
4,10,30,68.54.72.236,Guilty Trex
5,14,30,2.125.18.131,GamingConsolesUK
6,13,30,174.111.70.2,PcFreak30
7,15,38,99.242.38.212,RAMI/MOE
8,16,78,74.128.149.51,griff112 and Cytosine51
9,17,78,108.84.78.118,Crudit


#### Q1: Is this table different from other previous loaded table? YES
#### Q2: Is this table similar to any table in the generic Booter database schema? YES
    - This table looks like: friendsenemies
    
#### Q3: Are there modifications required? 
    - On the table name: 
        o friend-list ->friendsenemies
    - On the column type: 
        o    
    - On the column name:
        o host -> ip
        o name -> note
        o uid -> userid 
    - Split columns:
        o 
    - Add required columns:
        o type = friend      

### Modifying table: 'friend-list'

In [104]:
# Loading the column converting columns with predefined functions
df_friendsenemies = pd.read_csv(read_inserted_table(dumpfile, tablename),delimiter=",",error_bad_lines=False)

# Converters                                     

# Changing names of columns
df_friendsenemies.rename(inplace=True,columns = {'host':'ip',
                                                'name':'note',
                                                'uid':'userid'})

# Creating empty columns (with "" [for future string] or np.nan [for future float])
df_friendsenemies['type'] = 'friend'


# Showing some lines after adapt the table
df_friendsenemies.head(10)

,id,userid,ip,note,type
0,7,30,68.82.250.68,BoomFireProductions,friend
1,29,30,31.110.102.168,TehModdah,friend
2,5,30,86.27.70.195,Cobby,friend
3,114,78,98.213.93.116,DeathGodz,friend
4,10,30,68.54.72.236,Guilty Trex,friend
5,14,30,2.125.18.131,GamingConsolesUK,friend
6,13,30,174.111.70.2,PcFreak30,friend
7,15,38,99.242.38.212,RAMI/MOE,friend
8,16,78,74.128.149.51,griff112 and Cytosine51,friend
9,17,78,108.84.78.118,Crudit,friend


## 6.4. Table: 'memberships'

### How this table looks like without modification?

In [105]:
tablename='memberships'

pd.read_csv(read_inserted_table(dumpfile, tablename),delimiter=",",error_bad_lines=False).head(10)

,id,name,price,recurring,boot
0,bronzemonthly,Bronze Monthly,5.00,1,300
1,silvermonthly,Silver Monthly,10.00,1,450
2,goldmonthly,Gold Monthly,15.00,1,600
3,platinummonthly,Platinum Monthly,20.00,1,1000
4,bronzelifetime,Bronze Lifetime,20.00,0,300
5,silverlifetime,Silver Lifetime,30.00,0,450
6,goldlifetime,Gold Lifetime,40.00,0,600
7,platinumlifetime,Platinum Lifetime,50.00,0,1000
8,ultimatemonthly,Ultimate Monthly,50.00,1,3600
9,ultimatelifetime,Ultimate Lifetime,200.00,0,3600


#### Q1: Is this table different from other previous loaded table? YES
#### Q2: Is this table similar to any table in the generic Booter database schema? YES
    - This table looks like: plans
#### Q3: Are there modifications required? 
    - On the table name: 
        o memberships => plans
    - On the column type: 
        o 
    - On the column name:
        o id -> planid
        o name -> planname 
        o boot -> maxboottime 
        o recurring -> concurrecy
    - Add required columns:
        o  plandescr
    - Split columns:
        o 

### Modifying table: 'memberships'

In [106]:
# Loading the correct table name
df_plans = pd.read_csv(read_inserted_table(dumpfile, tablename),delimiter=",",error_bad_lines=False)

# Converting columns

# Changing column names
df_plans.rename(inplace=True, columns = {'id':'planid',
                                         'name':'planname',
                                         'boot':'maxboottime',
                                         'recurring':'concurrency'})

# Adding missing columns (with "" [for future string] or np.nan [for future float])
df_plans['plandescr'] = ''

###Showing some lines after adapt the table
df_plans.head(10)

,planid,planname,price,concurrency,maxboottime,plandescr
0,bronzemonthly,Bronze Monthly,5.00,1,300,
1,silvermonthly,Silver Monthly,10.00,1,450,
2,goldmonthly,Gold Monthly,15.00,1,600,
3,platinummonthly,Platinum Monthly,20.00,1,1000,
4,bronzelifetime,Bronze Lifetime,20.00,0,300,
5,silverlifetime,Silver Lifetime,30.00,0,450,
6,goldlifetime,Gold Lifetime,40.00,0,600,
7,platinumlifetime,Platinum Lifetime,50.00,0,1000,
8,ultimatemonthly,Ultimate Monthly,50.00,1,3600,
9,ultimatelifetime,Ultimate Lifetime,200.00,0,3600,


## 6.5. Table: 'news'

### How this table looks like without modification?

In [108]:
tablename='news'

pd.read_csv(read_inserted_table(dumpfile, tablename),delimiter=",",error_bad_lines=False).head(10)

,id,message,date
0,1,All members please resign up with same informa...,1341892848
1,2,Hey everyone got a new thread on HF \r\nhttp:/...,1342732597
2,3,Skype resolver will be u soon i contacted DaL3...,1343431842


#### Q1: Is this table different from other previous loaded table? YES 
#### Q2: Is this table similar to any table in the generic Booter database schema? NO
    - This table looks like: 
#### Q3: Are there modifications required? 
    - On the table name: 
        o 
    - On the column type: 
        o  
    - On the column name:
        o 
    - Add required columns:
        o 
    - Split columns:
        o 

## 6.6. Table: 'paypal-purchases'

### How this table looks like without modification?

In [111]:
tablename='paypal-purchases'

pd.read_csv(read_inserted_table(dumpfile, tablename),delimiter=",",error_bad_lines=False).head(10)

,id,uid,email,item,amount
0,1,245,ybarra.debra@yahoo.com,Bronze Monthly,5.0
1,2,250,cody.d.hudson@Gmail.com,Bronze Monthly,5.0
2,3,252,garycooke12345@hotmail.co.uk,Bronze Monthly,5.0
3,4,255,peder.carola@mail.se,Bronze Monthly,5.0
4,5,259,anthony.pacheco13@gmail.com,Bronze Monthly,5.0
5,6,260,tannerhacks@live.com,Bronze Monthly,5.0
6,7,262,tunny1234@live.dk,Bronze Monthly,5.0
7,8,264,corrodedmagnum@yahoo.com,Bronze Lifetime,20.0
8,9,265,djelias92@gmail.com,Bronze Monthly,5.0
9,10,268,johnellison89@sky.com,Bronze Monthly,5.0


#### Q1: Is this table different from other previous loaded table? YES 
#### Q2: Is this table similar to any table in the generic Booter database schema? YES
    - This table looks like: payments
#### Q3: Are there modifications required? 
    - On the table name: 
        o paypal-purchases -> payments
    - On the column type: 
        o  
    - On the column name:
        o uid -> userid
        o email -> paymentemail
        o amount -> amountpaid
    - Add required columns:
        o username
        o planid
        o tid
        o date
    - Split columns:
        o 

### Modifying table: 'paypal-purchases'

In [112]:
# Loading the correct table name
df_payments = pd.read_csv(read_inserted_table(dumpfile, tablename),delimiter=",",error_bad_lines=False)

# Converting columns

# Changing column names
df_payments.rename(inplace=True, columns = {'uid':'userid',
                                           'email': 'paymentemail',
                                           'amount': 'amountpaid'})

# Showing some lines after adapt the tabledf_plans['maxboottime'] = np.nan
df_payments['username'] = ''
df_payments['planid'] = np.nan
df_payments['tid'] = np.nan
df_payments['date'] = ''

# Showing some lines after adapt the table
df_payments.head(10)

,id,userid,paymentemail,item,amountpaid,username,planid,tid,date
0,1,245,ybarra.debra@yahoo.com,Bronze Monthly,5.0,,NaN,NaN,
1,2,250,cody.d.hudson@Gmail.com,Bronze Monthly,5.0,,NaN,NaN,
2,3,252,garycooke12345@hotmail.co.uk,Bronze Monthly,5.0,,NaN,NaN,
3,4,255,peder.carola@mail.se,Bronze Monthly,5.0,,NaN,NaN,
4,5,259,anthony.pacheco13@gmail.com,Bronze Monthly,5.0,,NaN,NaN,
5,6,260,tannerhacks@live.com,Bronze Monthly,5.0,,NaN,NaN,
6,7,262,tunny1234@live.dk,Bronze Monthly,5.0,,NaN,NaN,
7,8,264,corrodedmagnum@yahoo.com,Bronze Lifetime,20.0,,NaN,NaN,
8,9,265,djelias92@gmail.com,Bronze Monthly,5.0,,NaN,NaN,
9,10,268,johnellison89@sky.com,Bronze Monthly,5.0,,NaN,NaN,


## 6.7. Table: 'shells'

### How this table looks like without modification?

In [113]:
tablename='shells'

pd.read_csv(read_inserted_table(dumpfile, tablename),delimiter=",",error_bad_lines=False).head(10)

,id,url,special
0,1,http://206.72.206.150/ass.php,0
1,2,http://46.166.147.121/udp.php,0
2,3,http://46.166.147.121/ssyn.php,0
3,4,http://46.166.147.121/csyn.php,0


#### Q1: Is this table different from other previous loaded table? YES 
#### Q2: Is this table similar to any table in the generic Booter database schema? YES
    - This table looks like: webshells
#### Q3: Are there modifications required? 
    - On the table name: 
        o 
    - On the column type: 
        o 
    - On the column name:
        o 
    - Add required columns:
        o status
        o lastchecked 
        o attacktype
    - Split columns:
        o 

### Modifying table: 'shells'

In [114]:
# Loading the correct table name
df_webshells = pd.read_csv(read_inserted_table(dumpfile, tablename),delimiter=",",error_bad_lines=False)

# Converting columns

# Changing names of columns

# Creating empty columns (with "" [for future string] or np.nan [for future float])
df_webshells['status']=''
df_webshells['lastchecked']=''
df_webshells['attacktype']=''

# Showing some lines after adapt the table
df_webshells.head(10)

,id,url,special,status,lastchecked,attacktype
0,1,http://206.72.206.150/ass.php,0,,,
1,2,http://46.166.147.121/udp.php,0,,,
2,3,http://46.166.147.121/ssyn.php,0,,,
3,4,http://46.166.147.121/csyn.php,0,,,


## 6.8. Table: 'specialmemberships'

### How this table looks like without modification?

In [115]:
tablename='specialmemberships'

pd.read_csv(read_inserted_table(dumpfile, tablename),delimiter=",",error_bad_lines=False).head(10)

,id,uid,membership,price,coupon
0,2,1,bronzemonthly,4,avpjfunmpmjev56yq1
1,4,1,ultimatelifetime,0,vhx2dziwi6514d426w


#### Q1: Is this table different from other previous loaded table? YES 
#### Q2: Is this table similar to any table in the generic Booter database schema? NO
    - This table looks like: 
#### Q3: Are there modifications required? 
    - On the table name: 
        o 
    - On the column type: 
        o 
    - On the column name:
        o 
    - Add required columns:
        o 
    - Split columns:
        o 

## 6.9. Table: 'users'

### How this table looks like without modification?

In [117]:
tablename='users'

pd.read_csv(read_inserted_table(dumpfile, tablename),delimiter=",",error_bad_lines=False).head(10)

,id,username,password,salt,email,ip,lastact,token,usergroup,subdays,lastboot,cooldown,banned,membership,lastlogin,tos
0,1,EKNOZ,3jujb4ortkp10ft0zx83a97ebcf19013d5111732c420e6...,3jujb4ortkp10ft0zx,eknoz@live.com,24.92.81.234,1.343630e+09,x9749il5citu,1,0,1.343629e+09,50,0,ultimatelifetime,1.343629e+09,agree
1,2,Connerrr,td31jl6lo1kcgchx8s836d282c92e191fa93948e48c37b...,td31jl6lo1kcgchx8s,conz2012@hotmail.co.uk,2.100.40.59,1.343652e+09,9mch1bbdsagu,0,12,1.343631e+09,180,0,bronzemonthly,1.343650e+09,agree
2,3,TaylorGang,wjlq25zxikgy65wgvh352b8928dfc34f267cf4592dd1e3...,wjlq25zxikgy65wgvh,tgsgaming@live.com,208.38.66.32,1.343371e+09,ikepeyxnc284,0,18,1.343369e+09,60,0,silvermonthly,1.343366e+09,agree
3,4,jztmanyl,r00bj0uit72ebf34nlec8a34e9002da6fbd854e1818e5c...,r00bj0uit72ebf34nl,maxtheawsome@hotmail.se,95.209.100.228,1.342104e+09,8v0uh66htyhc,0,5,1.342104e+09,200,0,bronzemonthly,1.342104e+09,agree
4,5,Delshad,s3h9vxt57ir95o0ppdb02518639936171d3c569a9db898...,s3h9vxt57ir95o0ppd,habnu@hotmail.dk,87.57.129.218,1.343631e+09,tygmaa6rmzyw,0,15,1.343626e+09,290,0,bronzemonthly,1.343631e+09,agree
5,6,ihasnousername,m52txbzxkm3uuitvj8856b6aaa2dda3e19331aad7401cf...,m52txbzxkm3uuitvj8,jztmanyl@gmail.com,95.209.100.228,1.343639e+09,e4a6selpi6p7,0,0,1.343639e+09,100,0,platinumlifetime,1.343639e+09,agree
6,7,syjuntan,3r1xoh3cdqieagc1k71ddf93f9ff99d1f110b7a3b14839...,3r1xoh3cdqieagc1k7,max.nylander@live.se,95.209.100.228,1.342104e+09,b3ub4l83eqfh,0,7,NaN,0,0,bronzemonthly,1.342104e+09,agree
7,8,zestia,j8u7goc9bywr1ujpohea5966e75fdff1e6936bd9390a0b...,j8u7goc9bywr1ujpoh,zestiahd@gmail.com,188.114.162.177,1.343571e+09,ut6sxoewj78l,0,0,1.343571e+09,100,0,bronzelifetime,1.343571e+09,agree
8,9,rangefluid,tvmsc7j9gupubmaaok22452a5d7b5ce95f7cd2e8e9e1e2...,tvmsc7j9gupubmaaok,steelersfan4377@yahoo.com,99.72.194.104,1.343667e+09,pvv5j4s4y2vt,0,0,1.343183e+09,300,0,bronzelifetime,1.343667e+09,agree
9,10,tc4waza,bc8m286mywj5dkh52zce777cf9f6db95d1736d1da884fa...,bc8m286mywj5dkh52z,velez@gmx.us,98.253.169.154,1.343450e+09,f2b6iqgii77u,0,0,1.343450e+09,100,0,bronzelifetime,1.343450e+09,agree


#### Q1: Is this table different from other previous loaded table? YES 
#### Q2: Is this table similar to any table in the generic Booter database schema? YES
    - This table looks like: users
#### Q3: Are there modifications required? 
    - On the table name: 
        o 
    - On the column type: 
        o 
    - On the column name:
        o email -> useremail 
        o membership -> planid
    - Add required columns:
        o userid
        o expire
    - Split columns:
        o df_logins['username']=df_users['username']
        o df_logins['userip']=df_users['ip']
        o df_logins['date']=timestamp2datetime(df_users['lastlogin'])

### Modifying table: 'paypal_payments'

In [119]:
# Loading the correct table name
df_users = pd.read_csv(read_inserted_table(dumpfile, tablename),delimiter=",",error_bad_lines=False)

# Converting columns

# Changing names of columns
df_users.rename(inplace=True, columns = {'email':'useremail',
                                        'membership': 'planid'})

# Creating empty columns (with "" [for future string] or np.nan [for future float])
df_users['userid']=np.nan
df_users['expire']=''

# Showing some lines after adapt the table
df_users.head(10)

,id,username,password,salt,useremail,ip,lastact,token,usergroup,subdays,lastboot,cooldown,banned,planid,lastlogin,tos,userid,expire
0,1,EKNOZ,3jujb4ortkp10ft0zx83a97ebcf19013d5111732c420e6...,3jujb4ortkp10ft0zx,eknoz@live.com,24.92.81.234,1.343630e+09,x9749il5citu,1,0,1.343629e+09,50,0,ultimatelifetime,1.343629e+09,agree,NaN,
1,2,Connerrr,td31jl6lo1kcgchx8s836d282c92e191fa93948e48c37b...,td31jl6lo1kcgchx8s,conz2012@hotmail.co.uk,2.100.40.59,1.343652e+09,9mch1bbdsagu,0,12,1.343631e+09,180,0,bronzemonthly,1.343650e+09,agree,NaN,
2,3,TaylorGang,wjlq25zxikgy65wgvh352b8928dfc34f267cf4592dd1e3...,wjlq25zxikgy65wgvh,tgsgaming@live.com,208.38.66.32,1.343371e+09,ikepeyxnc284,0,18,1.343369e+09,60,0,silvermonthly,1.343366e+09,agree,NaN,
3,4,jztmanyl,r00bj0uit72ebf34nlec8a34e9002da6fbd854e1818e5c...,r00bj0uit72ebf34nl,maxtheawsome@hotmail.se,95.209.100.228,1.342104e+09,8v0uh66htyhc,0,5,1.342104e+09,200,0,bronzemonthly,1.342104e+09,agree,NaN,
4,5,Delshad,s3h9vxt57ir95o0ppdb02518639936171d3c569a9db898...,s3h9vxt57ir95o0ppd,habnu@hotmail.dk,87.57.129.218,1.343631e+09,tygmaa6rmzyw,0,15,1.343626e+09,290,0,bronzemonthly,1.343631e+09,agree,NaN,
5,6,ihasnousername,m52txbzxkm3uuitvj8856b6aaa2dda3e19331aad7401cf...,m52txbzxkm3uuitvj8,jztmanyl@gmail.com,95.209.100.228,1.343639e+09,e4a6selpi6p7,0,0,1.343639e+09,100,0,platinumlifetime,1.343639e+09,agree,NaN,
6,7,syjuntan,3r1xoh3cdqieagc1k71ddf93f9ff99d1f110b7a3b14839...,3r1xoh3cdqieagc1k7,max.nylander@live.se,95.209.100.228,1.342104e+09,b3ub4l83eqfh,0,7,NaN,0,0,bronzemonthly,1.342104e+09,agree,NaN,
7,8,zestia,j8u7goc9bywr1ujpohea5966e75fdff1e6936bd9390a0b...,j8u7goc9bywr1ujpoh,zestiahd@gmail.com,188.114.162.177,1.343571e+09,ut6sxoewj78l,0,0,1.343571e+09,100,0,bronzelifetime,1.343571e+09,agree,NaN,
8,9,rangefluid,tvmsc7j9gupubmaaok22452a5d7b5ce95f7cd2e8e9e1e2...,tvmsc7j9gupubmaaok,steelersfan4377@yahoo.com,99.72.194.104,1.343667e+09,pvv5j4s4y2vt,0,0,1.343183e+09,300,0,bronzelifetime,1.343667e+09,agree,NaN,
9,10,tc4waza,bc8m286mywj5dkh52zce777cf9f6db95d1736d1da884fa...,bc8m286mywj5dkh52z,velez@gmx.us,98.253.169.154,1.343450e+09,f2b6iqgii77u,0,0,1.343450e+09,100,0,bronzelifetime,1.343450e+09,agree,NaN,


In [125]:
df_logins=pd.DataFrame()
df_logins['username']=df_users['username']
df_logins['userip']=df_users['ip']
df_logins['date']=timestamp2datetime(df_users['lastlogin'])

df_logins['id']=''
df_logins['userid']=''

df_logins.head()

,username,userip,date,id,userid
0,EKNOZ,24.92.81.234,2012-07-30 06:23:29,,
1,Connerrr,2.100.40.59,2012-07-30 12:10:52,,
2,TaylorGang,208.38.66.32,2012-07-27 05:18:06,,
3,jztmanyl,95.209.100.228,2012-07-12 14:41:47,,
4,Delshad,87.57.129.218,2012-07-30 06:46:56,,


<h2 align='center'>======================================================================
<br>Final step of the manual part
======================================================================</h2>

## 7. Adding missing tables accordingly to our generic Booter database 

In [126]:
#df_attacks= pd.DataFrame(columns=['id','userid','username','targetip','targeturl','duration','port','type','date'])
df_blacklist=pd.DataFrame(columns=['id','ip','note'])
#df_friendsenemies=pd.DataFrame(columns=['id','ip','note','userid','type'])
df_gateways=pd.DataFrame(columns=['email'])
#df_logins=pd.DataFrame(columns=['id','userid','username','userip','date'])
#df_payments=pd.DataFrame(columns=['id','userid','username','amountpaid','paymentemail','planid','tid','date'])
#df_plans=pd.DataFrame(columns=['planid','planname','plandescr','price','maxboottime','concurrency'])
df_servers=pd.DataFrame(columns=['id','ip'])
#df_settings=pd.DataFrame(columns=['url','sitename','siteemail'])
#df_users=pd.DataFrame(columns=['userid','username','useremail','password','expire','plan'])      
#df_webshells=pd.DataFrame(columns=['id','url','status','lastchecked','attacktype'])

<h1 align="center">
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!<br>
STEP 2: Data Enrichment<br>
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!</h1> 

### * Depending of the size of the data, this part can take HOURS. I tested for both small and big datasets and it worked. Be pacient. This will pay-off!

## * Libraries needed to retrieve information from external databases

In [ ]:
import subprocess
import cfscrape
from lxml import etree
import os.path
import random
import time 

## * Function to enrich IP addresseswith AS information and country 

In [ ]:
# THANKS TO: team-cymru.org
def iptoasn(iplist_teamcymruformat_filelocation,outputfile):
    cat = subprocess.Popen(['cat', iplist_teamcymruformat_filelocation], 
                            stdout=subprocess.PIPE)
    
    netcat = subprocess.Popen(['netcat', 'whois.cymru.com', '43'],
                              stdin=cat.stdout,
                              stdout=outputfile)
    time.sleep(3) #for some reason the poll does not work! This was the way to overcome the waiting time.
    
    return netcat.stdout      

## * Function to check if an IP address was Tor node in a given moment

In [ ]:
# THANKS TO: exonerator.torproject.org
def WasTorNode(ip, date ):
    url="https://exonerator.torproject.org/?ip="+ip+"&timestamp="+date
    scraper = cfscrape.create_scraper()
    scraped_html=scraper.get(url).content    
    html_tree = etree.HTML(scraped_html)
    result=html_tree.xpath("//h3[@class='panel-title']/text()") # I was looking for <h3 class="panel-title">Result is positive</h3>
    tor_node=True if result == ['Result is positive'] else False
    return tor_node 
# 'date' MUST BE formated as: Year-month-day (2016-03-31)

## * Discovering the middle date of the database

In [ ]:
try:
    try:
        middle_date=(min(df_attacks['date'])+((max(df_attacks['date'])-min(df_attacks['date']))/2))
        raise
    except Exception:
        pass

    try:
        middle_date=(min(df_logins['date'])+((max(df_logins['date'])-min(df_logins['date']))/2))
        raise
    except Exception:
        pass

    try:
        middle_date=(min(df_payments['date'])+((max(df_payments['date'])-min(df_payments['date']))/2))
        raise
    except:
        pass
except Exception:
    print("There is no date in the entire dataset")

date_tor_check = middle_date.strftime('%Y-%m-%d')
date_iptoasn_lookup= str(middle_date)
print(date_tor_check)

## * Preparing to Perform IP to ASN info

In [ ]:
df_logins['middledate']=date_iptoasn_lookup
df_attacks['middledate']=date_iptoasn_lookup
df_friendsenemies['middledate']=date_iptoasn_lookup
df_blacklist['middledate']=date_iptoasn_lookup

## 2.1.1  Lookup IP to ASN info of table: logins

In [ ]:
if (os.path.exists('enrichments/logins_iptoasn_out')== False):
    logins_iptoasn_in = open('enrichments/logins_iptoasn_in', 'w+')
    logins_iptoasn_in.write('begin\nverbose\n')
    df_logins[['userip','middledate']].drop_duplicates().to_csv(logins_iptoasn_in,header=False,index=False,sep="\t") 
    logins_iptoasn_in.write('end')
    logins_iptoasn_in.close()

    logins_iptoasn_out = open('logins_iptoasn_out', 'w+')
    iptoasn('logins_iptoasn_in',logins_iptoasn_out)
    logins_iptoasn_out.close()
else:
    print("You already performed the lookup for this table")

In [ ]:
df_logins_iptoasn = pd.read_csv('enrichments/logins_iptoasn_out',\
                                skiprows=1,\
                             delimiter="\s+\|\s",\
                             names = ['asn', 'ip', 'bgp_prefix', 'country','registry','info_date','info_request','as_name'])

df_logins_extended= pd.merge(df_logins,
                              df_logins_iptoasn,
                              how = 'left',
                              left_on = 'userip',
                              right_on = 'ip')

# Changing name of columns to avoid misunderstandings
df_logins_extended.rename(columns={'asn':'srcasn', 
                                   'ip':'srcip', 
                                   'bgp_prefix':'srcbgp_prefix', 
                                   'country':'srccountry' ,
                                   'registry':'srcregistry',
                                   'info_date':'srcinfo_date',
                                   'info_request':'srcinfo_request'},
                         inplace=True)


df_logins_extended.head(5)

## 2.1.2  Lookup IP to ASN info of table: attacks

In [ ]:
if (os.path.exists('enrichments/attacks_iptoasn_out')== False):
    attacks_iptoasn_in = open('enrichments/attacks_iptoasn_in', 'w+')
    attacks_iptoasn_in.write('begin\nverbose\n')
    df_attacks[['targetip','middledate']].drop_duplicates().to_csv(attacks_iptoasn_in,header=False,index=False,sep="\t") 
    attacks_iptoasn_in.write('end')
    attacks_iptoasn_in.close()

    attacks_iptoasn_out = open('attacks_iptoasn_out', 'w+')
    iptoasn('attacks_iptoasn_in',attacks_iptoasn_out)
    attacks_iptoasn_out.close()
else:
    print("You already performed the lookup for this table")

In [ ]:
df_attacks_iptoasn = pd.read_csv('enrichments/attacks_iptoasn_out',\
                                skiprows=1,\
                             delimiter="\s+\|\s",\
                             names = ['asn', 'ip', 'bgp_prefix', 'country','registry','info_date','info_request','as_name'])

# Merging the iptoasn with the queried column
df_attacks_extended= pd.merge(df_attacks,
                              df_attacks_iptoasn,
                              how = 'left',
                              left_on = 'targetip',
                              right_on = 'ip')

# Changing name of columns to avoid misunderstandings
df_attacks_extended.rename(columns={'asn':'targetasn', 
                                   'ip_y':'targetip', 
                                   'bgp_prefix':'targetbgp_prefix', 
                                   'country_y':'targetcountry' ,
                                   'registry':'targetregistry',
                                   'info_date':'targetinfo_date',
                                   'info_request':'targetinfo_request'},
                         inplace=True)
df_attacks_extended.head(5)

## 2.1.3  Lookup IP to ASN info of table: friendsenemies

In [ ]:
if (os.path.exists('enrichments/friendsenemies_iptoasn_out')== False):
    friendsenemies_iptoasn_in = open('enrichments/friendsenemies_iptoasn_in', 'w+')
    friendsenemies_iptoasn_in.write('begin\nverbose\n')
    df_friendsenemies[['ip','middledate']].drop_duplicates().to_csv(friendsenemies_iptoasn_in,header=False,index=False,sep="\t") 
    friendsenemies_iptoasn_in.write('end')
    friendsenemies_iptoasn_in.close()

    friendsenemies_iptoasn_out = open('friendsenemies_iptoasn_out', 'w+')
    iptoasn('friendsenemies_iptoasn_in',friendsenemies_iptoasn_out)
    friendsenemies_iptoasn_out.close()
else:
    print("You already performed the lookup for this table.")

In [ ]:
df_friendsenemies_iptoasn = pd.read_csv('enrichments/friendsenemies_iptoasn_out',\
                                skiprows=1,\
                             delimiter="\s+\|\s",\
                             names = ['asn', 'ip', 'bgp_prefix', 'country','registry','info_date','info_request','as_name'])

# Merging the iptoasn with the queried column
df_friendsenemies_extended= pd.merge(df_friendsenemies,
                              df_friendsenemies_iptoasn,
                              how = 'left',
                              left_on = 'ip',
                              right_on = 'ip')

# Changing name of columns to avoid misunderstandings
df_friendsenemies_extended.rename(columns={'asn':'friendsenemiesasn', 
                                   'ip':'friendsenemiesip', 
                                   'bgp_prefix':'friendsenemiesbgp_prefix', 
                                   'country':'friendsenemiescountry' ,
                                   'registry':'friendsenemiesregistry',
                                   'info_date':'friendsenemiesinfo_date',
                                   'info_request':'friendsenemiesinfo_request',
                                   'as_name': 'friendsenemiesas_name'},
                         inplace=True)

## 2.1.4  Lookup IP to ASN info of table: blacklist

In [ ]:
if (os.path.exists('enrichments/blacklist_iptoasn_out')== False):
    blacklist_iptoasn_in = open('enrichments/blacklist_iptoasn_in', 'w+')
    blacklist_iptoasn_in.write('begin\nverbose\n')
    df_blacklist[['ip','middledate']].drop_duplicates().to_csv(blacklist_iptoasn_in,header=False,index=False,sep="\t") 
    blacklist_iptoasn_in.write('end')
    blacklist_iptoasn_in.close()

    blacklist_iptoasn_out = open('blacklist_iptoasn_out', 'w+')
    iptoasn('blacklist_iptoasn_in',blacklist_iptoasn_out)
    blacklist_iptoasn_out.close()
else:
    print("You already performed the lookup for this table.")

In [ ]:
df_blacklist_iptoasn = pd.read_csv('enrichments/blacklist_iptoasn_out',\
                                skiprows=1,\
                             delimiter="\s+\|\s",\
                             names = ['asn', 'ip', 'bgp_prefix', 'country','registry','info_date','info_request','as_name'])

# Merging the iptoasn with the queried column
df_blacklist_extended= pd.merge(df_blacklist,
                              df_blacklist_iptoasn,
                              how = 'left',
                              left_on = 'ip',
                              right_on = 'ip')

# Changing name of columns to avoid misunderstandings
df_blacklist_extended.rename(columns={'asn':'blacklistasn', 
                                   'ip':'blacklistip', 
                                   'bgp_prefix':'blacklistbgp_prefix', 
                                   'country':'blacklistcountry' ,
                                   'registry':'blacklistregistry',
                                   'info_date':'blacklistinfo_date',
                                   'info_request':'blacklistinfo_request',
                                   'as_name': 'blacklistas_name'},
                         inplace=True)
df_blacklist_extended.head(5)

## 2.2.1. Check if IP was a TOR node for table: login

In [ ]:
date_tor_check

In [ ]:
if len(df_logins['userip'].unique())<1200:
    if (os.path.exists('enrichments/logins_torcheck')== False):
        print("Note: it can take a while to finish...")
        logins_torcheck = open('logins_torcheck', 'w+')
        for i in df_logins['userip'].unique():
            wasTor=WasTorNode(i,date_tor_check)
            print(i, wasTor, file=logins_torcheck)
    #         print(i, wasTor) #DEBUGING =D
            time.sleep(random.randint(1,3)) #adding some random sleep time
            logins_torcheck.flush()

        logins_torcheck.close()
    else:
        print("The IP addresses from this table were already checked.")
else:
        print("Aborted!!! It will take more than one hour to analyse!")

In [ ]:
df_logins_torcheck = pd.read_csv('enrichments/logins_torcheck',\
                          delimiter="\s+",\
                          names = ['userip', 'tor'])

## 2.2.2. Check if IP was a TOR node for table: attacks

In [ ]:
if len(df_attacks['targetip'].unique())<1200:
    if (os.path.exists('enrichments/attacks_torcheck')== False):
        print("Note: it can take a while to finish...",len(df_attacks['targetip'].unique())*3,"seconds (in the worst case).")

        attacks_torcheck = open('attacks_torcheck', 'w+')

        for i in df_attacks['targetip'].unique():
            wasTor=WasTorNode(i,date_tor_check)
            print(i, wasTor, file=attacks_torcheck)
            print(i, wasTor) #DEBUGING =D
            time.sleep(random.randint(1,3)) #adding some random sleep time
            attacks_torcheck.flush()

        attacks_torcheck.close()
    else:
        print("The IP addresses from this table were already checked.") 
else:
        print("Aborted!!! It will take more than one hour to analyse!")

In [ ]:
df_attacks_torcheck = pd.read_csv('enrichments/attacks_torcheck',\
                          delimiter="\s+",\
                          names = ['targetip', 'tor'])

## 2.2.3. Check if IP was a TOR node for table: friendsenemies

In [ ]:
if len(df_friendsenemies['ip'].unique()) <1200:
    if (os.path.exists('enrichments/friendsenemies_torcheck')== False):
        print("Note: it can take a while to finish...",len(df_friendsenemies['ip'].unique())*3,"seconds (in the worst case).")

        friendsenemies_torcheck = open('enrichments/friendsenemies_torcheck', 'w+')

        for i in df_friendsenemies['ip'].unique():
            wasTor=WasTorNode(i,date_tor_check)
            print(i, wasTor, file=friendsenemies_torcheck)
        #     print(i, wasTor) #DEBUGING =D
            time.sleep(random.randint(1,3)) #adding some random sleep time
            friendsenemies_torcheck.flush()

        friendsenemies_torcheck.close()
    else:
        print("The IP addresses from this table were already checked.") 
else:
        print("Aborted!!! It will take more than one hour to analyse!")

In [ ]:
df_friendsenemies_torcheck = pd.read_csv('enrichments/friendsenemies_torcheck',\
                          delimiter="\s+",\
                          names = ['ip', 'tor'])

## 2.2.4. Check if IP was a TOR node for table: blacklist

In [ ]:
if len(df_blacklist['ip'].unique()) < 1200:
    if (os.path.exists('enrichments/blacklist_torcheck')== False):
        print("Note: it can take a while to finish...",len(df_blacklist['ip'].unique())*3,"seconds (in the worst case).")

        blacklist_torcheck = open('enrichments/blacklist_torcheck', 'w+')

        for i in df_blacklist['ip'].unique():
            wasTor=WasTorNode(i,date_tor_check)
            print(i, wasTor, file=blacklist_torcheck)
    #         print(i, wasTor) #DEBUGING =D
            time.sleep(random.randint(1,3)) #adding some random sleep time
            blacklist_torcheck.flush()

        blacklist_torcheck.close()
    else:
        print("The IP addresses from this table were already checked.") 
else:
        print("Aborted!!! It will take more than one hour to analyse!")

In [ ]:
df_blacklist_torcheck = pd.read_csv('enrichments/blacklist_torcheck',\
                          delimiter="\s+",\
                          names = ['ip', 'tor'])

## * Relation between Attack dates and Login dates

In [ ]:
def nearestDate(base_date, date_list):
    nearest={}
    for date in date_list:
        if (base_date.timestamp() - date.timestamp())>=0:
            nearest[base_date.timestamp() - date.timestamp()]= date
    return nearest[min(nearest.keys())]

In [ ]:
#This is the TOTAL number records to be checks!!!!
len(df_attacks)

In [ ]:
df_attacks['nearestlogin']=""
df_attacks['nearestlogin']=pd.to_datetime(df_attacks['nearestlogin'])
    
if len(df_attacks)>0 and len(df_logins)>0:
    #When was the last login of the user that performed attacks
    df_attacks['nearestlogin']=""
    df_attacks['nearestlogin']=pd.to_datetime(df_attacks['nearestlogin'])

    for index, row in df_attacks.head(100).iterrows():
        intermediate_df= df_logins[df_logins['username']==row['username']]
        nearestlogindate= nearestDate(row['date'],intermediate_df['date'])
        df_attacks.set_value(index, 'nearestlogin', nearestlogindate)

        #DEBUGGING
        if index % 1000 == 0:
            print(index,": +1000 records analysed")

In [ ]:
if len(df_attacks['nearestlogin'].value_counts()) >1:
    df_attacks_and_logins = pd.merge(df_attacks_extended,
                                     df_logins_extended,
                                     how = 'left',
                                     left_on = ['username','nearestlogin'],
                                     right_on = ['username','date'])
else:
    df_attacks_and_logins=""

<h2 align="center">
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!<br>
STEP 3: Automatic Analysis
<br>
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!</h3> 

## * Libraries that I use to plot figs

In [ ]:
import matplotlib.pyplot as plt
from matplotlib_venn import *
%matplotlib inline

import seaborn as sns
import itertools

plt.style.use('seaborn-muted')
# plt.rcParams['font.family'] = 'serif'
# plt.rcParams['font.size'] = 12

import warnings
warnings.filterwarnings('ignore')

## A.1. Number of records per table (part of the generic Booter database schema)

In [ ]:
len(df_users),len(df_logins),len(df_attacks),len(df_payments),len(df_settings),len(df_gateways), len(df_friendsenemies),len(df_blacklist),len(df_webshells),len(df_servers),len(df_plans)

## A.2. Number of users, customers, attackers, and their intersection

In [ ]:
if len(set(df_users['userid'].unique())) > 1:
    users_set=set(df_users['userid'].unique())
else:
    users_set=set(df_users['username'].unique())
    
if len(set(df_logins['userid'].unique())) > 1:
    userslogin_set=set(df_logins['userid'].unique())
else:
    userslogin_set=set(df_logins['username'].unique())

if len(set(df_payments['userid'].unique())) > 1:
    customers_set=set(df_payments['userid'].unique())
else:
    customers_set=set(df_payments['username'].unique())

if len(set(df_attacks['userid'].unique())) > 1:
    attackers_set=set(df_attacks['userid'].unique())
else:
    attackers_set=set(df_attacks['username'].unique())

intersec_customers_attacker=pd.Series(list(customers_set.intersection(attackers_set)))
intersec_users_customers=pd.Series(list(users_set.intersection(customers_set)))
intersec_users_attackers=pd.Series(list(users_set.intersection(attackers_set)))
intersec_users_customers_attackers=pd.Series(list(users_set.intersection(customers_set).intersection(attackers_set)))

In [ ]:
len(users_set),len(userslogin_set),len(customers_set),len(attackers_set)

In [ ]:
fig = plt.figure(figsize=(4,3))
fig.suptitle('Users, Customers & Attackers', fontsize=14)

ax = plt.subplot2grid((1,1), (0,0))

venn=venn3(ax=ax,subsets = {'001':len(attackers_set)-len(intersec_customers_attacker)-len(intersec_users_attackers)+len(intersec_users_customers_attackers), 
                            '010':len(customers_set)-len(intersec_users_customers)-len(intersec_customers_attacker)+len(intersec_users_customers_attackers), 
                            '011':len(intersec_customers_attacker)-len(intersec_users_customers_attackers),
                            '100':len(users_set)-len(intersec_users_customers)-len(intersec_users_attackers)+len(intersec_users_customers_attackers),
                            '101':len(intersec_users_attackers)-len(intersec_users_customers_attackers),
                            '110':len(intersec_users_customers)-len(intersec_users_customers_attackers),
                            '111':len(intersec_users_customers_attackers)},\
          set_labels = ('Users', 'Customers','Attackers'),\
          alpha=1)
try:
    venn.get_patch_by_id('100').set_color('#404096')
except:
    print("*Users set is empty!")  
    
try:
    venn.get_patch_by_id('110').set_color('#DEA73A')
except:
    print("*Customers set is empty!")   

try:
    venn.get_patch_by_id('001').set_color('#D92120')
except:
    print("*Attackers set is empty!")

fig.show()
# fig.savefig('../figs/timeseries_attacks.eps', format='eps', dpi=1200) 

## A.3. Distribution of login times per user

In [ ]:
if len(df_logins['userid'].value_counts()) > 0:
    num_distinct_logins_per_user=df_logins['userid'].value_counts()
else:
    num_distinct_logins_per_user=df_logins['username'].value_counts()

freq_distinct_logins_per_user=num_distinct_logins_per_user.value_counts()
cum_dist_user_logins = np.linspace(0.,1.,len(num_distinct_logins_per_user))
cdf_user_logins = pd.Series(cum_dist_user_logins, index=num_distinct_logins_per_user.sort_values())

In [ ]:
if len(num_distinct_logins_per_user)>0:
    fig = plt.figure(figsize=(6, 3))
    fig.suptitle('* Distribution of Login Times by Users:', fontsize=14, y=1.05,x=0.35)
    
    #Plot CDF
    ax1 = plt.subplot2grid((1,2), (0,0))
    ax1 = cdf_user_logins.plot(ax=ax1,lw=2, drawstyle='steps',legend=False)
    ax1.set_xlabel("# logins")
    ax1.set_ylabel("CDF of Users")
    ax1.set_ylim(0,1.05)
    ax1.set_title("")
    plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45)

    
    ax2 = plt.subplot2grid((1,2), (0,1))
    freq_distinct_logins_per_user.plot(ax=ax2,kind='pie', 
                                       labels=None, 
                                       legend=False,
                                       startangle=270,
#                                        colors=sns.color_palette()
                                       )
    ax2.set_ylabel("")
    
    
    
    ####
    # For Pie Chart Better Legend box
    porcent = (100.*freq_distinct_logins_per_user.values)/(freq_distinct_logins_per_user.values.sum())
    labels = ['{0} - {1:1.2f} %'.\
              format(i,j) for i,j in zip(freq_distinct_logins_per_user.index, porcent)]
    # To Sort Legend (defaulf: keeps the same order)
#     patches, labels, dummy =  zip(*sorted(zip(patches, labels, freq_distinct_logins_per_user.values),
#                                           key=lambda x: x[2],
#                                           reverse=True))
    legend_show_top=10
    ax2.legend(ax2.patches[0:legend_show_top], 
               labels[0:legend_show_top], 
               bbox_to_anchor=(1.5, 1.),
               fontsize=10)
    ####
    
    fig.tight_layout()
    fig.show()
    fig.savefig('figs/login_times.eps', bbox_inches='tight',format='eps', dpi=1200)
else:
    print("Unfortunately, there is no data available!")


## 5.3. Number of Users that Login via TOR

In [ ]:
len(df_logins_torcheck[df_logins_torcheck['tor']==True]['userip'].value_counts())

## 5.4. Number of Distinct IP addresses by Users

In [ ]:
if len(df_logins.groupby(['userid','userip']).size().reset_index()['userid'].value_counts()) >0:
    num_distinct_ips_per_user=df_logins.groupby(['userid','userip']).size().reset_index()['userid'].value_counts()
else:
    num_distinct_ips_per_user=df_logins.groupby(['username','userip']).size().reset_index()['username'].value_counts()
    
freq_distinct_ips_per_user=num_distinct_ips_per_user.value_counts()
cum_dist_user_ips = np.linspace(0.,1.,len(num_distinct_ips_per_user))
cdf_user_ips = pd.Series(cum_dist_user_ips, index=num_distinct_ips_per_user.sort_values())

In [ ]:
if len(num_distinct_ips_per_user)>0:
    fig = plt.figure(figsize=(6, 3))
    fig.suptitle('* Distribution of Distinct IP address used by Users:', fontsize=14, y=1.05, x=0.4)

    ax1 = plt.subplot2grid((1,2), (0,0))
    ax1 = cdf_user_ips.plot( ax=ax1,lw=2, drawstyle='steps',legend=False)
    ax1.set_xlabel("# IPs")
    ax1.set_ylabel("CDF of Users")
    ax1.set_ylim(0,1.05)
    plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45)
    
    ax2 = plt.subplot2grid((1,2), (0,1))
    freq_distinct_ips_per_user.plot(ax=ax2,kind='pie',
                                    labels=None,legend=False,
                                       startangle=270,
#                                        colors=sns.color_palette()
                                       )
    ax2.set_ylabel("")
    ####
    # For Pie Chart Better Legend box
    porcent = (100.*freq_distinct_ips_per_user.values)/(freq_distinct_ips_per_user.values.sum())
    labels = ['{0} - {1:1.2f} %'.\
              format(i,j) for i,j in zip(freq_distinct_ips_per_user.index, porcent)]
    # To Sort Legend (defaulf: keeps the same order)
#     patches, labels, dummy =  zip(*sorted(zip(patches, labels, freq_distinct_logins_per_user.values),
#                                           key=lambda x: x[2],
#                                           reverse=True))
    legend_show_top=10
    ax2.legend(ax2.patches[0:legend_show_top], 
               labels[0:legend_show_top], 
               bbox_to_anchor=(1.5, 1.),
               fontsize=10)
    ####
    
    fig.tight_layout()
    fig.show()
    fig.savefig('figs/num_ips_by_users.eps', bbox_inches='tight',format='eps', dpi=1200)
else:
    print("Unfortunately, there is no data available!")


## 5.5. Number of Payments by Users

In [ ]:
if len(df_payments['userid'].value_counts())>0:
    num_distinct_payments_per_user=df_payments['userid'].value_counts()
else:
    num_distinct_payments_per_user=df_payments['username'].value_counts()

freq_distinct_payments_per_user=num_distinct_payments_per_user.value_counts()
cum_dist_user_payments = np.linspace(0.,1.,len(num_distinct_payments_per_user))
cdf_user_payments = pd.Series(cum_dist_user_payments, index=num_distinct_payments_per_user.sort_values())

In [ ]:
if len(num_distinct_payments_per_user)>0:
    fig = plt.figure(figsize=(6, 3))
    fig.suptitle('* Distribution of Number of Payments by Users:', fontsize=14, y=1.05, x=0.4)

    ax1 = plt.subplot2grid((1,2), (0,0))
    ax1 = cdf_user_payments.plot( ax=ax1,lw=2, drawstyle='steps',legend=False)
    ax1.set_xlabel("# Payment")
    ax1.set_ylabel("CDF of Users")
    ax1.set_ylim(0,1.05)
    plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45)
    
    ax2 = plt.subplot2grid((1,2), (0,1))
    freq_distinct_payments_per_user.plot(ax=ax2,kind='pie', 
                                         labels=None,legend=False,
                                         startangle=270)
    ax2.set_ylabel("")
    ####
    # For Pie Chart Better Legend box
    porcent = (100.*freq_distinct_payments_per_user.values)/(freq_distinct_payments_per_user.values.sum())
    labels = ['{0} - {1:1.2f} %'.\
              format(i,j) for i,j in zip(freq_distinct_payments_per_user.index, porcent)]
    # To Sort Legend (defaulf: keeps the same order)
#     patches, labels, dummy =  zip(*sorted(zip(patches, labels, freq_distinct_logins_per_user.values),
#                                           key=lambda x: x[2],
#                                           reverse=True))
    legend_show_top=10
    ax2.legend(ax2.patches[0:legend_show_top], 
               labels[0:legend_show_top], 
               bbox_to_anchor=(1.55, 1.),
               fontsize=10)
    ####
    fig.tight_layout()
    fig.show()
    fig.savefig('figs/payments_distribution.eps', bbox_inches='tight',format='eps', dpi=1200)
else:
    print("Unfortunately, there is no data available!")


## 5.6. Total Amount of Money Earned

In [ ]:
if len(df_payments['amountpaid']) >0:
    total_earned=df_payments['amountpaid'].values.sum()
    'US$ {:,.2f}'.format(float(total_earned))
else:
    print("Unfortunately, there is no data available!")

## 5.7. Amount of Money Paid by Users

In [ ]:
num_distinct_payments_money_per_user=df_payments['amountpaid'].value_counts()
freq_distinct_payments_money_per_user=num_distinct_payments_money_per_user.value_counts()
cum_dist_user_payments_money = np.linspace(0.,1.,len(num_distinct_payments_money_per_user))
cdf_user_payments_money = pd.Series(cum_dist_user_payments_money, index=num_distinct_payments_money_per_user.sort_values())

In [ ]:
if len(num_distinct_payments_money_per_user)>0:
    fig = plt.figure(figsize=(6, 3))
    fig.suptitle('* Distribution of Money Payments by Users:', fontsize=14, y=1.05, x=0.4)

    ax1 = plt.subplot2grid((1,2), (0,0))
    ax1 = cdf_user_payments_money.plot( ax=ax1,lw=2, drawstyle='steps',legend=False)
    ax1.set_xlabel("$")
    ax1.set_ylabel("CDF of Users")
    ax1.set_ylim(0,1.05)
    plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45)

    ax2 = plt.subplot2grid((1,2), (0,1))
    freq_distinct_payments_money_per_user.plot(ax=ax2,kind='pie', 
                                               labels=None,legend=False,
                                               startangle=270)
    ax2.set_ylabel("")
    ####
    # For Pie Chart Better Legend box
    porcent = (100.*freq_distinct_payments_money_per_user.values)/(freq_distinct_payments_money_per_user.values.sum())
    labels = ['${0:1.2f} - {1:1.2f} %'.\
              format(i,j) for i,j in zip(freq_distinct_payments_money_per_user.index, porcent)]
    # To Sort Legend (defaulf: keeps the same order)
#     patches, labels, dummy =  zip(*sorted(zip(patches, labels, freq_distinct_logins_per_user.values),
#                                           key=lambda x: x[2],
#                                           reverse=True))
    legend_show_top=10
    ax2.legend(ax2.patches[0:legend_show_top], 
               labels[0:legend_show_top], 
               bbox_to_anchor=(1.6, 1.),
               fontsize=10)
    ####
    fig.tight_layout()
    fig.show()
    fig.savefig('figs/distribution_amount_paid.eps', bbox_inches='tight',format='eps', dpi=1200)
else:
    print("Unfortunately, there is no data available!")


## 5.8. Countries from where Users Access Booters

In [ ]:
logins_country_distribution_sorted = df_logins_iptoasn['country'].value_counts(sort=True,ascending=True)
logins_country_distribution = df_logins_iptoasn['country'].value_counts()

In [ ]:
if len(logins_country_distribution)>0:
    fig = plt.figure(figsize=(6, 3))
    fig.suptitle('* Users Accessing from Countries:', fontsize=14, y=1.05, x=0.28)

    ax1 = plt.subplot2grid((1,2), (0,0))
    logins_country_distribution_sorted.plot(ax=ax1,kind='barh')
    ax1.set_ylabel("# access")
    ax1.set_xlabel("Country")
    plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45)

    ax2 = plt.subplot2grid((1,2), (0,1))
    logins_country_distribution.plot(ax=ax2,kind='pie', 
                                     labels=None,legend=False,
                                     startangle=270)
    ax2.set_ylabel("")
    ####
    # For Pie Chart Better Legend box
    porcent = (100.*logins_country_distribution.values)/(logins_country_distribution.values.sum())
    labels = ['{0} - {1:1.2f} %'.\
              format(i,j) for i,j in zip(logins_country_distribution.index, porcent)]
    # To Sort Legend (defaulf: keeps the same order)
#     patches, labels, dummy =  zip(*sorted(zip(patches, labels, freq_distinct_logins_per_user.values),
#                                           key=lambda x: x[2],
#                                           reverse=True))
    legend_show_top=10
    ax2.legend(ax2.patches[0:legend_show_top], 
               labels[0:legend_show_top], 
               bbox_to_anchor=(1.55, 1.),
               fontsize=10)
    ####
    fig.tight_layout()
    fig.show()
    fig.savefig('figs/user_countries.eps', bbox_inches='tight',format='eps', dpi=1200)
else:
    print("Unfortunately, there is no data available!")

## 5.9. Countries of Blacklisted IPs

In [ ]:
blacklist_country_distribution=df_blacklist_iptoasn['country'].value_counts()
blacklist_country_distribution_sorted=df_blacklist_iptoasn['country'].value_counts(sort=True,ascending=True)

In [ ]:
if len(blacklist_country_distribution)>0:
    fig = plt.figure(figsize=(6, 3))
    fig.suptitle('* Countries of blacklisted IPs', fontsize=14, y=1.05, x=0.28)

    ax1 = plt.subplot2grid((1,2), (0,0))
    blacklist_country_distribution_sorted.plot(ax=ax1,kind='barh')
    ax1.set_ylabel("# Access")
    ax1.set_xlabel("Country")
    plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45)

    ax2 = plt.subplot2grid((1,2), (0,1))
    blacklist_country_distribution.plot(ax=ax2,kind='pie', 
                                        labels=None,legend=False,
                                        startangle=270)
    ax2.set_ylabel("")
    ####
    # For Pie Chart Better Legend box
    porcent = (100.*blacklist_country_distribution.values)/(logins_country_distribution.values.sum())
    labels = ['{0} - {1:1.2f} %'.\
              format(i,j) for i,j in zip(blacklist_country_distribution.index, porcent)]
    # To Sort Legend (defaulf: keeps the same order)
#     patches, labels, dummy =  zip(*sorted(zip(patches, labels, freq_distinct_logins_per_user.values),
#                                           key=lambda x: x[2],
#                                           reverse=True))
    legend_show_top=10
    ax2.legend(ax2.patches[0:legend_show_top], 
               labels[0:legend_show_top], 
               bbox_to_anchor=(1.55, 1.),
               fontsize=10)
    ####
    fig.tight_layout()
    fig.show()
    fig.savefig('figs/blacklist_countries.eps', bbox_inches='tight',format='eps', dpi=1200)
else:
    print("Unfortunately, there is no data available!")

## 5.10. Countries of Target IPs

In [ ]:
attacks_country_distribution=df_attacks_iptoasn['country'].value_counts()

In [ ]:
if len(attacks_country_distribution)>0:
    fig = plt.figure(figsize=(6, 3))
    fig.suptitle('* Countries of target IPs', fontsize=14, y=1.05, x=0.28)

    ax1 = plt.subplot2grid((1,2), (0,0))
    attacks_country_distribution.plot(ax=ax1,kind='bar')
    ax1.set_ylabel("# Access")
    ax1.set_xlabel("Country")
    plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45)

    ax2 = plt.subplot2grid((1,2), (0,1))
    attacks_country_distribution.plot(ax=ax2,kind='pie', 
                                      labels=None,legend=False,
                                      startangle=270)
    ax2.set_ylabel("")
    ####
    # For Pie Chart Better Legend box
    porcent = (100.*attacks_country_distribution.values)/(attacks_country_distribution.values.sum())
    labels = ['{0} - {1:1.2f} %'.\
              format(i,j) for i,j in zip(attacks_country_distribution.index, porcent)]
    # To Sort Legend (defaulf: keeps the same order)
#     patches, labels, dummy =  zip(*sorted(zip(patches, labels, freq_distinct_logins_per_user.values),
#                                           key=lambda x: x[2],
#                                           reverse=True))
    legend_show_top=10
    ax2.legend(ax2.patches[0:legend_show_top], 
               labels[0:legend_show_top], 
               bbox_to_anchor=(1.55, 1.),
               fontsize=10)
    ####
    fig.tight_layout()
    fig.show()
    fig.savefig('figs/attack_countries_distribution.eps',bbox_inches='tight', format='eps', dpi=1200) 
else:
    print("Unfortunately, there is no data available!")

## 5.12. Attacks on Same Target

In [ ]:
num_attacks_on_sametarget=df_attacks['targetip'].value_counts()

freq_num_attacks_on_sametarget=num_attacks_on_sametarget.value_counts()
cum_num_attacks_on_sametarget = np.linspace(0.,1.,len(num_attacks_on_sametarget))
cdf_num_attacks_on_sametarget = pd.Series(cum_num_attacks_on_sametarget, index=num_attacks_on_sametarget.sort_values())

In [ ]:
if len(num_attacks_on_sametarget)>0:
    fig = plt.figure(figsize=(6, 3))
    fig.suptitle('* Attacks on Same Targets:', fontsize=14, y=1.05,x=0.28)

    ax1 = plt.subplot2grid((1,2), (0,0))
    ax1 = cdf_num_attacks_on_sametarget.plot( ax=ax1,lw=2, drawstyle='steps',legend=False)
    ax1.set_xlabel("# IPs")
    ax1.set_ylabel("CDF of Users")
    ax1.set_ylim(0,1.05)
    plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45)

    ax2 = plt.subplot2grid((1,2), (0,1))
    freq_num_attacks_on_sametarget.plot(ax=ax2,kind='pie',
                                        labels=None,legend=False,
                                        startangle=270)
    ax2.set_ylabel("")
    ####
    # For Pie Chart Better Legend box
    porcent = (100.*freq_num_attacks_on_sametarget.values)/(freq_num_attacks_on_sametarget.values.sum())
    labels = ['{0} - {1:1.2f} %'.\
              format(i,j) for i,j in zip(freq_num_attacks_on_sametarget.index, porcent)]
    # To Sort Legend (defaulf: keeps the same order)
#     patches, labels, dummy =  zip(*sorted(zip(patches, labels, freq_distinct_logins_per_user.values),
#                                           key=lambda x: x[2],
#                                           reverse=True))
    legend_show_top=10
    ax2.legend(ax2.patches[0:legend_show_top], 
               labels[0:legend_show_top], 
               bbox_to_anchor=(1.55, 1.),
               fontsize=10)
    ####
    fig.tight_layout()

    fig.show()
    fig.savefig('figs/attacks_on_same_target.eps', bbox_inches='tight',format='eps', dpi=1200)
else:
    print("Unfortunately, there is no data available!")

## 5.11. Attacks per day (timeseries) 

In [ ]:
if len(df_attacks)>0:
    attack_timeseries=df_attacks.set_index(['date']).groupby(pd.TimeGrouper(freq='D')).agg(['count'])['action']
    attack_mean_perday=attack_timeseries.mean()
    attack_median_perday=attack_timeseries.median()
else:
    attack_timeseries=""

In [ ]:
if len(attack_timeseries)>0:
    fig = plt.figure(figsize=(6,3))

    ax1 = plt.subplot2grid((1,1), (0,0), rowspan=2)
    attack_timeseries.plot(ax=ax1,
                           lw=2,
                           legend=False,
    #                        style='--'
                          )

    # X and Y Labels and Ticks
    ax1.set_xlabel("Time (bin=day)")
    ax1.set_ylabel("# Attacks")

    ax1.annotate(str(int(attack_median_perday[0]))+' (median)', 
                 (min(df_attacks['date']), attack_median_perday),
                 xytext=(350, -1), 
                 textcoords='offset points',
                 color='black', 
                 arrowprops=dict(arrowstyle='-|>',
                                 color='black'))
    fig.savefig('figs/attacks_timeseries.eps', bbox_inches='tight',format='eps', dpi=1200)

else:
    print("Unfortunately, there is no data available!")

## 5.12. Time Between Logins and Attacks

In [ ]:
if len(df_attacks_and_logins)>0:
    print("redo")
else:
    print("Unfortunately, there is no data available!")

## 5.13. Who attack whom? (users on the country level)

In [ ]:
if len(df_attacks_extended)>0 and len(df_logins_extended)>0:
    merged_attacks_logins = pd.merge(df_attacks_extended,
                                  df_logins_extended,
                                  how = 'left',
                                  left_on = 'date',
                                  right_on = 'date')[['targetcountry','srccountry']]

    who_against_whom = merged_attacks_logins.groupby(['targetcountry','srccountry'])\
                            .size()\
                            .reset_index()\
                            .pivot('srccountry','targetcountry',0)
else:
    who_against_whom =""

In [ ]:
if len(who_against_whom)>0:
    fig = plt.figure(figsize=(8,12))
    fig.suptitle('Countries of Target IPs', fontsize=14, y=.92)

    ax1 = plt.subplot2grid((1,1), (0,0))
    sns.set()
    sns.heatmap(who_against_whom,
                ax=ax1,
#                 cmap="YlGnBu",
    #             linewidths=.5,
    #             annot=True
                )

    ax1.set_ylabel("Source Country")
    ax1.set_xlabel("Attack Target Country")

    fig.show()
    fig.savefig('figs/who_attack_whom.eps', bbox_inches='tight',format='eps', dpi=1200)
else:
    print("Unfortunately, there is no data available!")

<h1 align="center">!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!<br>
FIFTH PART: Query Interface<br>
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!</h1> 

In [ ]:
from ipywidgets import widgets, interact, Dropdown,HTML
from IPython.display import display,clear_output

In [ ]:
def search_by_userid_submit(sender):
    clear_output()
    print("Searching by id =",search_by_userid.value)
    
    if len(df_users[df_users['userid']== int(search_by_userid.value)])>0:
        display(df_users[df_logins['userid']== int(search_by_userid.value)])
    
    if len(df_logins_extended[df_logins_extended['userid']== int(search_by_userid.value)])>0:
        display(df_logins_extended[df_logins_extended['userid']== int(search_by_userid.value)])
    
    if len(df_attacks_extended[df_attacks_extended['userid']== int(search_by_userid.value)])>0:
        display(df_attacks_extended[df_attacks_extended['userid']== int(search_by_userid.value)])
    
    if len(df_payments[df_payments['userid']== int(search_by_userid.value)])>0:
        display(df_payments[df_payments['userid']== int(search_by_userid.value)])

def search_by_username_submit(sender):
    clear_output()
    print("Searching by username =",search_by_username.value,"\n")
    
    if len(df_users[df_users['username']== search_by_username.value])>0:
        print("Table df_users:")
        display(df_users[df_users['username']== search_by_username.value])
    
    if len(df_logins_extended[df_logins_extended['username']== search_by_username.value])>0:
        print("Table df_logins:")
        display(df_logins_extended[df_logins_extended['username']== search_by_username.value])

    if len(df_attacks_extended[df_attacks_extended['username']== search_by_username.value])>0:
        print("Table df_attacks:")
        display(df_attacks_extended[df_attacks_extended['username']== search_by_username.value])
    
    if len(df_payments[df_payments['username']== search_by_username.value])>0:
        print("Table df_payments:")
        display(df_payments[df_payments['username']== search_by_username.value])
        
def search_by_ip_submit(sender):
    clear_output()
    print("Searching by IP address =",str(search_by_ip.value),"\n")
    
    if len(df_logins_extended[df_logins_extended['userip']== str(search_by_ip.value)])>0:
        print("Table df_logins (as ATTACKER:")
        display(df_logins_extended[df_logins_extended['userip']== str(search_by_ip.value)])
     
    if len(df_attacks_extended[df_attacks_extended['targetip']== str(search_by_ip.value)])>0:
        print("Table df_attacks (as TARGET):")
        display(df_attacks_extended[df_attacks_extended['targetip']== str(search_by_ip.value)])  
      
       
def search_by_asn_submit(sender):
    clear_output()
    print("Searching by Autonomous System Number (ASN) =",search_by_asn.value,"\n")
    
    if len(df_logins_extended[df_logins_extended['srcasn']== int(search_by_asn.value)])>0:
        print("Table df_logins (as ATTACKER):")
        display(df_logins_extended[df_logins_extended['srcasn']== int(search_by_asn.value)])
    
    if len(df_attacks_extended[df_attacks_extended['targetasn']== str(search_by_asn.value)])>0:
        print("Table df_attacks (as TARGET):")
        display(df_attacks_extended[df_attacks_extended['targetasn']== str(search_by_asn.value)])

    if len(df_attacks_extended[df_attacks_extended['targetasn']== str(search_by_asn.value)])>0:
        print("Table df_attacks (as TARGET):")
        display(df_attacks_extended[df_attacks_extended['targetasn']== str(search_by_asn.value)])

        
country_list=pd.read_csv('https://raw.githubusercontent.com/datasets/country-list/master/data.csv',delimiter=",",error_bad_lines=False)
def search_by_country_submit():
    country_code=country_list[country_list['Name']==search_by_country.value]['Code'].values[0]
    clear_output()
    print("Searching by Country =",search_by_country.value,"\n")
    
    if len(df_logins_extended[df_logins_extended['srccountry']== country_code])>0:
        print("Table df_logins (as ATTACKER):")
        display(df_logins_extended[df_logins_extended['srccountry']== country_code])
    
    if len(df_attacks_extended[df_attacks_extended['targetcountry']== country_code])>0:
        print("Table df_attacks (as TARGET):")
        display(df_attacks_extended[df_attacks_extended['targetcountry']== country_code])
    

In [ ]:
display(HTML('<h3>*Search by (only) one field per time:</h3>'))

search_by_username = widgets.Text(description="username")
display(search_by_username)
search_by_username.on_submit(search_by_username_submit)

search_by_userid = widgets.Text(description="userid")
display(search_by_userid)
search_by_userid.on_submit(search_by_userid_submit)

search_by_ip = widgets.Text(description="IP")
display(search_by_ip)
search_by_ip.on_submit(search_by_ip_submit)

search_by_asn = widgets.Text(description="ASN")
display(search_by_asn)
search_by_asn.on_submit(search_by_asn_submit)

search_by_country = Dropdown(description="Country", options=country_list['Name'].tolist())
search_by_country.on_trait_change(search_by_country_submit, name="value")
display(search_by_country)